# Published pre-trained model

Publish the pre-trained model for comparrisons or other uses

In [ ]:
%cd /opt/app-root/src/

# run detect on images to dowload the model if not already present
!yolo predict model=yolov8n.pt source=datasets/training/valid/images exist_ok=True

In [ ]:
import os
import urllib3
from urllib3.exceptions import InsecureRequestWarning
from minio import Minio
from minio.commonconfig import Tags

# Disable SSL verification warning
urllib3.disable_warnings(InsecureRequestWarning)

# MinIO server endpoint and access credentials
endpoint_str = os.environ.get('AWS_S3_ENDPOINT')
minio_endpoint = endpoint_str.lstrip('https://')
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

# MinIO bucket information
bucket_name = 'flyingthings'
file_path = 'yolov8n.pt'
object_name = 'model_pretrained.pt'


# Initialize MinIO client with SSL verification disabled
client = Minio(minio_endpoint, access_key=access_key, secret_key=secret_key, secure=True, http_client=urllib3.PoolManager(cert_reqs='CERT_NONE'))
file_exists = False
try:
    # Get object information.
    result = client.stat_object(bucket_name, object_name)
    print(
        "last-modified: {0}, size: {1}".format(
            result.last_modified, result.size,
        ),
    )
    file_exists = True
except:
    file_exists = False
    
# Upload the file and set tags
if not file_exists:
    client.fput_object(bucket_name, object_name, file_path)
    print(f"File '{object_name}' uploaded to bucket '{bucket_name}'")

    # Upload the new file
    client.fput_object(bucket_name, object_name, file_path)



# Setup

Install dependencies and check PyTorch and GPU.

In [ ]:
print("setting up dependencies......")
!pip install ultralytics --quiet
print("**** setup complete ****")

# Pull latest pre-trained and custom models

Download the pre-trained and custom models to this environment

In [ ]:
%cd /opt/app-root/src

!pip install minio
import os
import minio
import urllib3

# Disable SSL certificate verification
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set up MinIO client with insecure HTTPS
endpoint_str = os.environ.get('AWS_S3_ENDPOINT')
minio_endpoint = endpoint_str.lstrip('https://')
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
minio_client = minio.Minio(
    endpoint=minio_endpoint,
    access_key=access_key,
    secret_key=secret_key,
    secure=True,
    http_client=urllib3.PoolManager(cert_reqs='CERT_NONE', assert_hostname=False)
)

# Set the bucket name and file name
bucket_name = os.environ.get('AWS_S3_BUCKET')
pretrained_name = 'model_pretrained.pt'
custom_name = 'model_custom.pt'

# Download the file
minio_client.fget_object(bucket_name, pretrained_name, pretrained_name)
print(f"The file '{pretrained_name}' has been downloaded successfully.")

minio_client.fget_object(bucket_name, custom_name, custom_name)
print(f"The file '{custom_name}' has been downloaded successfully.")


# Infer with pretrained model
Run object detection with the pre-trained model

In [ ]:
%cd /opt/app-root/src/

# run detect on images to dowload the model if not already present
!yolo predict model=model_pretrained.pt source=datasets/training/valid/images exist_ok=True

# Show images
Show the bounding box images generated from last inferrence run

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, HTML

print("Please wait...")
# Set the directory path where the image files are located
directory = 'runs/detect/predict'

# Get all image files in the directory
image_files = [file for file in os.listdir(directory) if file.endswith(('.jpg', '.jpeg', '.png'))]

# Create subplots for displaying the images
num_images = len(image_files)
num_rows = (num_images - 1) // 3 + 1
fig, axes = plt.subplots(num_rows, 3, figsize=(12, num_rows*4))

# Generate links for each image
for i, file in enumerate(image_files):
    image_path = os.path.join(directory, file)
    img = mpimg.imread(image_path)
    ax = axes[i // 3, i % 3]  # Adjust indexing for 3 columns
    ax.imshow(img)
    ax.axis('off')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()


# Infer with custom model
Run object detection with the custom trained model

In [ ]:
%cd /opt/app-root/src/

# run detect on images to dowload the model if not already present
!yolo predict model=model_custom.pt source=datasets/training/valid/images exist_ok=True

# Show images
Show the bounding box images generated from last inferrence run

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, HTML

print("Please wait...")
# Set the directory path where the image files are located
directory = 'runs/detect/predict'

# Get all image files in the directory
image_files = [file for file in os.listdir(directory) if file.endswith(('.jpg', '.jpeg', '.png'))]

# Create subplots for displaying the images
num_images = len(image_files)
num_rows = (num_images - 1) // 3 + 1
fig, axes = plt.subplots(num_rows, 3, figsize=(12, num_rows*4))

# Generate links for each image
for i, file in enumerate(image_files):
    image_path = os.path.join(directory, file)
    img = mpimg.imread(image_path)
    ax = axes[i // 3, i % 3]  # Adjust indexing for 3 columns
    ax.imshow(img)
    ax.axis('off')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()
